# Configure packages and filepaths

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import os
import math
from os.path import join
from pathlib import Path
import pandas as pd
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from shapely.geometry import shape



In [3]:
# Filepaths

# Get the absolute path to the project directory
# Which is one directory above notebooks/
ABS_DIR = os.path.abspath(Path(os.getcwd()).parents[0])
# Get raw data directory
FR = join(ABS_DIR, 'data', 'raw')
# Get interim data directory
FI = join(ABS_DIR, 'data', 'interim')
# Get processed data directory
FP = join(ABS_DIR, 'data', 'processed')

# Directories for exposure, vulnerability (vuln) and 
# administrative reference files
EXP_DIR_R = join(FR, 'exposure')
VULN_DIR_R = join(FR, 'vuln')
REF_DIR_R = join(FR, 'ref')

In [11]:
# Constants (could be replaced by config files or user input)
# County fips (list for some scalability)
FIPS = ['42101']

# FEMA "chunk" size for API
CHUNK_FEMA = 1000

# Exposure Data

In [5]:
# Prepare file directories
Path(EXP_DIR_R).mkdir(parents=True, exist_ok=True)

## National Structure Inventory

In [6]:
# Get the URL
url = "https://nsi.sec.usace.army.mil/nsiapi/structures"

# Loop through counties, 
# Get the data from the NSI API
# Store in dataframe
# Add to list
# Concat all the dfs

# List for NSI DFs
nsi_df_list = []

for fips in FIPS:
    # GET Request
    nsi_get = requests.get(url + '?fips=' + fips)
    
    # Temp data frame
    temp = pd.json_normalize(nsi_get.json()['features'])
    
    # Add to list
    nsi_df_list.append(temp)

# Concat
nsi = pd.concat(nsi_df_list, axis=0)

# Write to file
nsi.to_parquet(join(EXP_DIR_R, 'nsi.pqt'))

## Local Municipal Data

## NFIP Data

### Policies

In [8]:
# Get the URL for querying policies
url = "https://www.fema.gov/api/open/v2/FimaNfipPolicies?$"
# Get the URL for # policies that meet request
check = url + "inlinecount=allpages&$top=1&$select=id&$"


# Loop through counties, 
# Get the data from the Pols API
# Store in dataframe
# Add to list
# Concat all the dfs

# List for Pols DFs
pol_df_list = []

# NFIP API usage adapts R code here: https://docs.ropensci.org/rfema/
# And follows OpenFEMA guide: 
# https://www.fema.gov/about/openfema/working-with-large-data-sets#app-a

for fips in FIPS:
    # County endpoint
    c_end = "filter=countyCode%20eq%20%27" + fips + "%27"
    
    # First, get the total number of records
    records = requests.get(check + c_end)
    n_rec = pd.json_normalize(records.json())['metadata.count'][0]
    
    # Get iterations needed (1,000 record limit)
    iterations = math.ceil(n_rec / CHUNK_FEMA)
    
    # Now, download 1,000 records at a time and store in list
    # Loop through required iterations and keep appending policy 
    # data from the GET request to the pol_df_list
    for i in range(iterations):
        skip_str = "&$skip=" + str(i*CHUNK_FEMA)
    
        # GET Request
        pol_get = requests.get(url + c_end + skip_str)

        # Temp data frame
        temp = pd.json_normalize(pol_get.json()['FimaNfipPolicies'])

        # Add to list
        pol_df_list.append(temp)

# Concat
nfip_pol = pd.concat(pol_df_list, axis=0)

# Write to file
nfip_pol.to_parquet(join(EXP_DIR_R, 'nfip_pols.pqt'))

### Claims

In [12]:
# Get the URL for querying claimicies
url = "https://www.fema.gov/api/open/v2/FimaNfipClaims?$"
# Get the URL for # claimicies that meet request
check = url + "inlinecount=allpages&$top=1&$select=id&$"


# Loop through counties, 
# Get the data from the claims API
# Store in dataframe
# Add to list
# Concat all the dfs

# List for claims DFs
claim_df_list = []

# NFIP API usage adapts R code here: https://docs.ropensci.org/rfema/
# And follows OpenFEMA guide: 
# https://www.fema.gov/about/openfema/working-with-large-data-sets#app-a

for fips in FIPS:
    # County endpoint
    c_end = "filter=countyCode%20eq%20%27" + fips + "%27"
    
    # First, get the total number of records
    records = requests.get(check + c_end)
    n_rec = pd.json_normalize(records.json())['metadata.count'][0]
    
    # Get iterations needed (1,000 record limit)
    iterations = math.ceil(n_rec / CHUNK_FEMA)
    
    # Now, download 1,000 records at a time and store in list
    # Loop through required iterations and keep appending claimicy 
    # data from the GET request to the claim_df_list
    for i in range(iterations):
        skip_str = "&$skip=" + str(i*CHUNK_FEMA)
    
        # GET Request
        claim_get = requests.get(url + c_end + skip_str)

        # Temp data frame
        temp = pd.json_normalize(claim_get.json()['FimaNfipClaims'])

        # Add to list
        claim_df_list.append(temp)

# Concat
nfip_claim = pd.concat(claim_df_list, axis=0)

# Write to file
nfip_claim.to_parquet(join(EXP_DIR_R, 'nfip_claims.pqt'))

# Vulnerability Data

## Physical Vulnerability

## Social Vulnerability

# Administrative Reference Data